In [ ]:
import torch
import os
from transformers import (
    AutoTokenizer,AutoModelForSeq2SeqLM ,
    Seq2SeqTrainingArguments ,Seq2SeqTrainer ,
    DataCollatorForSeq2Seq
)

from datasets import Dataset
import pandas as pd

#os.environ["CUDA_VISIBLE_DEVICE"]="0"

print("Cuda disponibilité",torch.cuda.is_available())
print("GPU",torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu')

Cuda disponibilité False
GPU cpu


In [ ]:
model_tokenizer = AutoTokenizer.from_pretrained("google/bert2bert_L-24_wmt_de_en", pad_token="<pad>", eos_token="</s>", bos_token="<s>")

#Custom Tokenizer

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.trainers import UnigramTrainer
from tokenizers.processors import TemplateProcessing
uni_tokenizer=Tokenizer(Unigram())

tok_trainer=UnigramTrainer(special_tokens=["[UNK]","[CLS]","[SEP]","[PAD]","[MASK]"])

In [ ]:
uni_tokenizer.post_processor=TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]",1),("[SEP]",2)]
)

In [ ]:
!pip install datasets --upgrade

In [ ]:
from datasets import load_dataset
dataset=load_dataset("google/smol","smolsent__en_fon")

In [ ]:
len(dataset['train'][0:]['trg'])

863

In [ ]:
writer=open("fon_src_from_smol.txt","w")
for row in dataset['train'][0:]['trg']:
  writer.write(row+"\n")
writer.close()

In [ ]:
path_smol_fon=["fon_src_from_smol.txt"]
uni_tokenizer.train(path_smol_fon,tok_trainer)

In [ ]:
smol_fon_output=uni_tokenizer.encode('Ku do zanzan')
print(f"Tokens :{smol_fon_output.tokens}")

Tokens :['[CLS]', 'K', 'u ', 'do ', 'zan', 'zan', '[SEP]']


In [ ]:
uni_tokenizer.save("fn_tokenizer.json")

# CUSTOM TOKENIZER

In [ ]:
import json
with open("fn_tokenizer.json") as f:
  tokenizer_data=json.load(f)

if "model" in tokenizer_data and "vocab" in tokenizer_data["model"]:
  # Iterate through the list of vocab items to extract the tokens
  new_tokens = [item[0] for item in tokenizer_data["model"]["vocab"]]
else:
  print("Adapt")
  new_tokens=tokenizer_data["vocab"]
model_tokenizer.add_tokens(new_tokens)
model_tokenizer.save_pretrained("bert2bert-extended_2fn")

('bert2bert-extended_2fn/tokenizer_config.json',
 'bert2bert-extended_2fn/special_tokens_map.json',
 'bert2bert-extended_2fn/vocab.txt',
 'bert2bert-extended_2fn/added_tokens.json',
 'bert2bert-extended_2fn/tokenizer.json')

In [ ]:
text="Nɛ̀ azǎn towe ka cí gbɔn?"
#tokens=model_tokenizer.tokenize(text)
#ids=model_tokenizer.convert_tokens_to_ids(tokens)
#print("Text",text)
#print("Tokens",tokens)
#print("IDS",ids)

In [ ]:
custom_tokenizer=AutoTokenizer.from_pretrained("/content/bert2bert-extended_2fn")
tokens=custom_tokenizer.tokenize(text)
ids=custom_tokenizer.convert_tokens_to_ids(tokens)
print("Text",text)
print("Tokens",tokens)
print("IDS",ids)


Text Nɛ̀ azǎn towe ka cí gbɔn?
Tokens ['nɛ ', 'azan ', 'towe ', 'ka ', 'ci ', 'gbɔn', '?']
IDS [32301, 33052, 33365, 31950, 31950, 32327, 136]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("google/bert2bert_L-24_wmt_de_en")

pytorch_model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

In [ ]:
model.encoder.resize_token_embeddings(len(custom_tokenizer))
model.decoder.resize_token_embeddings(len(custom_tokenizer))
inputs=custom_tokenizer(text,return_tensors='pt')
outputs=model(**inputs)
print(outputs.last_hidden_state.shape)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


ValueError: You have to specify either input_ids or inputs_embeds

# end custom

In [ ]:
def add_tags(elt):
  elt['src']=f"<{elt['sl']}>" +elt['src']
  elt['trg']=f"<{elt['tl']}>" +elt['trg']
  return elt

dataset=dataset.map(add_tags)

Map:   0%|          | 0/863 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['trg'][:4]

["<fon>E nɔ ɖ'alɔ mi bɔ un nɔ w'azɔ ɖe e jlo mi gbɔn e lobo nɔ ze nukplɔnmɛ walɔ ce ɖo ga ɖopko ɔ ji xá nukplɔnkplɔn walɔ tóɖomɛtɔ lɛ tɔn.",
 '<fon>Akɔmkpikpán ɖaxó mɛ e jló na dó hwɛ mi lɛ e tɔn hɛn fíbi gbadó.',
 '<fon>Nu e ɖo dotɛ ɖɔ nuwɛ hwɛhwɛ ɖó jlɛmɛ ɔ‚ e hɛn ɔ è na gbɔ vìvì tahoun;',
 "<fon>Ye kpewu bo wa nu e gbé e nya lɛ e bǐ lobo sɔ jiɖiɖe dó mɛɖesunɔ wu sín gǔ ɖe ɖo te nu mɛɖelɛ n'a kpɔn ɖewu."]

In [ ]:
from transformers import PreTrainedTokenizerFast
fon_tokenizer=Tokenizer.from_file('/content/fn_tokenizer.json')
fon_compat_tokenizer=PreTrainedTokenizerFast(
    tokenizer_object=fon_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    bos_token="<s>",
    eos_token="</s>"
)
fon_compat_tokenizer.save_pretrained("/content/fon_tokenizer_hf")

('/content/fon_tokenizer_hf/tokenizer_config.json',
 '/content/fon_tokenizer_hf/special_tokens_map.json',
 '/content/fon_tokenizer_hf/tokenizer.json')

In [ ]:
dataset['train']['is_src_orig'][:3]

[True, True, True]

In [ ]:
from transformers import AutoTokenizer
fon_tokenizer_loaded=AutoTokenizer.from_pretrained('/content/fon_tokenizer_hf')
fon_tokenizer_loaded.add_tokens(["<eng>","<fon>"])
fon_tokenizer_loaded.save_pretrained("/content/fon_tokenizer_hf")

('/content/fon_tokenizer_hf/tokenizer_config.json',
 '/content/fon_tokenizer_hf/special_tokens_map.json',
 '/content/fon_tokenizer_hf/tokenizer.json')

In [ ]:
print("[UNK]" in fon_tokenizer.get_vocab())

True


config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [ ]:
source_lang = "en"
target_lang = "fon"
prefix = "translate English to Fon: "


def preprocess_function(examples):
    print("EX",examples)
    inputs = examples['src']
    targets = examples['trg']
    model_inputs = model_tokenizer(inputs, max_length=128, truncation=True,padding=True)
    #print(inputs)
    #print(targets)
    with fon_tokenizer_loaded.as_target_tokenizer():
        labels = fon_tokenizer_loaded(targets, max_length=128, truncation=True,padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_set = dataset['train'].map(preprocess_function, batched=True)

Map:   0%|          | 0/863 [00:00<?, ? examples/s]

EX {'sl': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
